# Maximum Subarray Sum
Given an array of integers, return the sum of the subarray with the largest sum.

**Example:**
```python
Input: nums = [3, 1, -6, 2, -1, 4, -9]
Output: 5
```
Explanation: subarray [2, -1, 4] has the largest sum of 5.

**Constraints:**
- The input array contains at least one element.

## Intuition

Brute force approaches to this problem involve calculating the sum of every possible subarray. This would take at least **O(n²)** time, where `n` denotes the length of the array.

The challenge with this problem lies in the presence of negative numbers. If the array consisted entirely of non-negative numbers, the answer would simply be the sum of the entire array.

To find the maximum sum given the presence of negative numbers, let's try keeping track of the sum of a contiguous subarray, starting at index `0`.

As this subarray expands and we add each number to the running sum, we'll need to decide whether to continue with the current subarray's sum or start a new subarray beginning with the current element. 

To understand how we might make such a decision, let's dive into an example.

### Example
Consider the following input array:
```python
[3, 1, -6, 2, -1, 4, -9]
```

The first two values of the array are positive, so we can continue expanding the current subarray by adding these to our sum (`curr_sum`), initialized at `0`:

```python
curr_sum += 3 → 3 curr_sum += 1 → 4
```

When we reach index `2`, we land on the first negative number (`-6`). Adding it to the current sum gives us a negative sum of `-2`.

What should we do now? If we restart the subarray at this point, the new subarray will start with a sum of `-6`, which is less than the current sum of `-2`.  
Therefore, it is better to continue with the current subarray for now.

At index `3`, we reach another important decision point:
- If we include `2` in the current subarray, its sum increases to `0`:  

```python
curr_sum += 2 → 0
```
- If we restart the subarray at this index, we begin a new subarray with a sum of `2`.

It's evident that the better choice is to start tracking the sum of a new subarray, beginning at index `3`.

### Key Observation

For each number in the array during this process, there are two choices:
1. **Continue**: Add the current number to the ongoing subarray sum (`curr_sum + num`).
2. **Restart**: Begin keeping track of a new subarray starting with the current number, with an initial sum of `num`.

The best choice is the larger of the two:

```python
curr_sum = max(curr_sum + num, num)
```


Now that we have a strategy to linearly track subarray sums, the only other thing to do is keep track of the largest value of `curr_sum` encountered. To do this, we update a variable `max_sum` whenever we encounter a larger `curr_sum` value. Then, `max_sum` will be the answer to the problem.

---

## Kadane's Algorithm

The algorithm described above is formally known as **Kadane's algorithm**.  
Although it may not seem like it, Kadane's algorithm is actually a **dynamic programming (DP)** algorithm.

What's interesting is that we didn’t explicitly detect and solve subproblems to come up with this algorithm, like we typically do in DP. Instead, we solved it by linearly keeping track of a subarray sum and making decisions along the way.

In [1]:
from typing import List

def maximum_subarray_sum(nums: List[int]) -> int:
    if not nums:
        return 0

    max_sum = current_sum = float('-inf')

    for num in nums:
        current_sum = max(current_sum + num, num)
        max_sum = max(current_sum, max_sum)

    return max_sum

### Complexity Analysis
The time complexity is O(n) because we iterate through each element of the input array once.

The space complexity is O(1).

## Intuition - Dynamic Programming (DP)

An important observation is that every possible subarray **ends at a certain index**. This means each index signifies the **end** of several subarrays, and one of them will have the **maximum subarray sum** (shortened to "max subarray" moving forward) ending at that index.

So, how can we find the **max subarray** that ends at each index?  

### Example
Consider the given array:

```python
[3, 1, -6, 2, -1, 4, -9]
```

One thing we **know for sure** is that the max subarray **ending at a specific index** will definitely **include** the value at that index.  
We just need to determine if there are any **elements to the left** that also contribute to this **max subarray**.

In other words, we need to find the **max subarray** that **ends right before the last index**.
```python
max_subarray(i - 1) + (-9)
```

### Key Decision

Another thing to consider is the possibility that `max_subarray(i - 1)` is **negative**.  
If it is, then adding it would **decrease** the sum.  
In this case, the best option is to **start a new subarray** with just `nums[i]`.

This leads to the recurrence relation:

```python
max_subarray(i) = max(max_subarray(i - 1) + nums[i], nums[i])
```

This shows an **optimal substructure**, where the **max subarray at index `i`** depends on the **max subarray at index `i - 1`**.

This suggests we can solve the problem using **Dynamic Programming (DP)**.

### DP Formula
Translating the above recurrence relation into a DP formula:

```python
dp[i] = max(dp[i - 1] + nums[i], nums[i])
```

---

## Base Case

The **simplest subproblem** occurs when we consider **only the first element** of the array.  
When there's only **one element**, there's only **one possible subarray**.

Thus, the base case is:

```python
dp[0] = nums[0]
```

---

## Populating the DP Array

With the base case established, we can now populate the rest of the **DP array**.  
Starting from index `1` and going up to index `n - 1`, we calculate:

```python
dp[i] = max(dp[i - 1] + nums[i], nums[i])
```

While populating the DP array, we also **keep track of the maximum value** in `dp[]`, called `max_sum`, which represents the **largest sum of any subarray** in the entire array.

By the time we finish populating `dp[]`, we can simply **return `max_sum`**.


In [2]:
from typing import List

def maximum_subarray_sum(nums: List[int]) -> int:
    n = len(nums)
    if n == 0:
        return 0

    dp = [0] * n

    dp[0] = nums[0]
    max_sum = dp[0]

    for i in range(1, n):
        dp[i] = max(dp[i - 1] + nums[i], nums[i])
        max_sum = max(max_sum, dp[i])
    
    return max_sum

### Complexity Analysis
The time complexity is O(n) because we iterate through each element of the input array once.

The space complexity is O(n) because we're maintaining a DP array that contains n elements.

### Optimization
An important thing to note is that in the DP solution, we only ever need to access the previous value of the DP array (at i - 1) to calculate the current value (at i). This means we don't need to store the entire DP array.

Instead, we can use a single variable to keep track of the current subarray sum and update this value to calculate the next subarray sum.

This approach reduces the space complexity to O(1).

In [3]:
from typing import List

def maximum_subarray_sum(nums: List[int]) -> int:
    n = len(nums)
    if n == 0:
        return 0

    current_sum = nums[0]
    max_sum = nums[0]

    for i in range(1, n):
        current_sum = max(nums[i], current_sum + nums[i])
        max_sum = max(max_sum, current_sum)
    
    return max_sum